In [ ]:
# !pip install mido
# !pip install python-rtmidi
# !pip install playsound
# !pip install PyObjC
# !pip install pygame

In [ ]:
import mido
from playsound import playsound 
from ipywidgets import Select
from pygame import mixer

In [ ]:
loops_folder = '/Users/favreau/Music/Loops/'

In [ ]:
import glob
import os

kits = dict()

folders = glob.glob(loops_folder + '*')
# print(folders)
for folder in folders:
    files = glob.glob(folder + '/*')
    base_name = os.path.basename(folder)
    sounds = list()
    for file in files:
        sounds.append(os.path.basename(file))
    kits[base_name] = sounds

In [ ]:
Select(options=kits)

In [ ]:
mido.get_ioport_names()

In [ ]:
alesis = mido.open_input('Alesis Strike MultiPad')

In [ ]:
def play_sound(file_name):
    mode = MODE_ONE_SHOT
    if mode == MODE_ONE_SHOT:
        playsound(filename, True)
    else:
        while True:
            playsound(filename, True)

In [ ]:
import threading

active_threads = dict()

MODE_ONE_SHOT = 0
MODE_LOOP = 1


class SoundThread (threading.Thread):

    def __init__(self, kit, sound_id, mode):      
        threading.Thread.__init__(self)
        self._kit = kit
        self._sound_id = sound_id
        self._mode = mode
        
    def __del__(self):
        print('Delete')
        
    def play_sound(self):
        if self._sound_id < len(kits[self._kit]):
            if self._sound_id < len(kits[self._kit]):
                file_name = loops_folder + '/' + self._kit + '/' + kits[self._kit][self._sound_id]
                print('Play %s' % file_name)
                playsound(file_name, block=False)
            else:
                print('Wrong sound')
        else:
            print('Wrong kit')
        
    def run(self):
        if self._mode == MODE_ONE_SHOT:
            ''' One shot '''
            self.play_sound()
        elif self._mode == MODE_LOOP:
            ''' Loop '''
            while True:
                self.play_sound()

# Alesis Pads
# 44 45 46
# 40 41 42
# 36 37 38

alesis_pads = dict()
alesis_pads[44] = 0
alesis_pads[45] = 1
alesis_pads[46] = 2
alesis_pads[40] = 3
alesis_pads[41] = 4
alesis_pads[42] = 5
alesis_pads[36] = 6
alesis_pads[37] = 7
alesis_pads[38] = 8


import random

bpm = 112
delay = 60.0 / bpm
print(delay)
i = 3
while i> 0:
    note = random.randint(0, 6) # alesis.receive().note
    if active_threads.get(note):
        print('Stop thread %d' % note)
        active_threads[note].terminate()
        active_threads[note] = None

    print('Start thread %d' % note)
    kit = 'Pacman'
    sound_id = note # alesis_pads[note]
    file_name = loops_folder + '/' + kit + '/' + kits[kit][sound_id]
    t = multiprocessing.Process(target=playsound, args=(file_name,))  
    t.start()
    import time
    time.sleep(delay)
    active_threads[note] = t
    i -= 1
    
for t in active_threads:
    print('Stop thread %d' % t)
    active_threads[t].terminate()

In [ ]:
import time
time.time_ns()